In [ ]:
!apt-get update
!sudo apt-get install cdo

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,183 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,583 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-update

In [ ]:
!pip install netCDF4
!pip install nctoolkit
!pip install cdo
!pip install udunitspy
!pip install cfunits
#!pip install cf-python
from google.colab import drive
drive.mount('/content/gdrive')
import netCDF4 as nc
import numpy as np
import seaborn as sns
import pandas as pd
from shapely.geometry import Polygon
import shapely
import matplotlib.pyplot as plt
import os
from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import nctoolkit as nct
#import cf

import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout, Input, BatchNormalization
from keras.objectives import MSE, MAE
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import gc
import sys
from cdo import *
cdo = Cdo()

     |████████████████████████████████| 102kB 7.5MB/s 
     |████████████████████████████████| 3.1MB 20.8MB/s 
     |████████████████████████████████| 6.5MB 30.6MB/s 
     |████████████████████████████████| 337kB 50.0MB/s 
     |████████████████████████████████| 15.8MB 36.2MB/s 
     |████████████████████████████████| 4.2MB 31.9MB/s 
     |████████████████████████████████| 8.9MB 20.0MB/s 
     |████████████████████████████████| 12.1MB 36.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 512kB 59.5MB/s 
     |████████████████████████████████| 215kB 57.9MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 225kB 40.6MB/s 
     |████████████████████████████████| 706kB 55.4MB/s 
     |████████████████████████████████| 112kB 53.3MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl siz

**Define global variables**

In [ ]:
lat_bnds, lon_bnds = [-10.7, 3.51], [28.5, 40.7]

In [ ]:
date_range = pd.date_range('2016-01-01', '2020-12-31')
date_range = date_range.strftime('%Y%m%d').to_list()

**TAMSAT**

In [ ]:
one = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/TAMSAT/bounding_box_daily/01-tamsatDaily.v3.1-1262304000-1622502000_28.417_41.194_-11.183_3.622.nc')
one = one.where(one.apply(np.isfinite)).fillna(np.nan)

In [ ]:
one.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT/dataex.nc')

In [ ]:
data = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT/dataex.nc')

In [ ]:
data.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
data = data.to_xarray()
data.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT_regridded')

**IMERG**

In [ ]:
list_datasets = []
for date in tqdm(date_range):
  dataset = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/3B-DAY-L.MS.MRG.3IMERG.' + date + '-S000000-E235959.V06.nc4')
  dataset.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
  dataset.to_nc('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/IMERG_test/' + date + 'test.nc')
  #dataset = dataset.to_xarray()
  #list_datasets.append(dataset)
#dataset = xr.concat(list_datasets, dim = 'time')
#dataset.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/IMERG_regridded')

In [ ]:
data_list =  []
for date in tqdm(date_range):
  dataset = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/IMERG_test/' + date + 'test.nc')
  data_list.append(dataset)

In [ ]:
one = xr.concat(data_list, dim = 'time')

In [ ]:
one.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/IMERG_regridded.nc')

Variable 'time' has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for 'time' and 'time_bnds' will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for 'time' before writing to a file.


**CMORPH**

In [ ]:
date_range = pd.date_range('2016-01-01', '2020-12-31')
date_range = date_range.strftime('%Y%m%d').to_list()

In [ ]:
list_datasets = []
for date in tqdm(date_range):
  dataset = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/CMORPH_daily_copy/daily_aggCMORPH_V1.0_ADJ_8km-30min_' + date + '.nc')
  dataset.rename({'Longitude':'lon', 'Latitude':'lat', 'precip':'Precipitation'})
  dataset.set_units({'lon':'degrees_east', 'lat':'degrees_north','Precipitation':'mm'})
  dataset.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
  dataset.to_nc(('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMOPRH_test/' + date + '_test.nc' ))
  #ds['time'] = [pd.to_datetime(date)]
  #list_datasets.append(ds)
#dataset2 = xr.concat(list_datasets, dim = 'time')
#dataset2.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMORPH_regridded.nc')

In [19]:
data_list =  []
for date in tqdm(date_range):
  dataset = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMOPRH_test/' + date + '_test.nc')
  data_list.append(dataset)

In [20]:
one = xr.concat(data_list, dim = 'time')

In [22]:
one.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMORPH_regridded.nc')

In [ ]:
xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMOPRH_test/20160101_test.nc')

<xarray.Dataset>
Dimensions:        (lat: 285, lon: 245, time: 1)
Coordinates:
  * time           (time) float64 0.0
  * lon            (lon) float32 28.5 28.55 28.6 28.65 ... 40.55 40.6 40.65 40.7
  * lat            (lat) float32 -10.7 -10.65 -10.6 -10.55 ... 3.35 3.4 3.45 3.5
Data variables:
    Precipitation  (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Sun May 23 18:54:04 2021: cdo -L -z zip_9 copy /tmp/nctoolk...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

In [ ]:
list_datasets[0]

FileNotFoundError: ignored

<xarray.Dataset>
Dimensions:        (lat: 285, lon: 245, time: 1)
Coordinates:
  * time           (time) datetime64[ns] 2016-01-31
  * lon            (lon) float32 28.5 28.55 28.6 28.65 ... 40.55 40.6 40.65 40.7
  * lat            (lat) float32 -10.7 -10.65 -10.6 -10.55 ... 3.35 3.4 3.45 3.5
Data variables:
    Precipitation  (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Sun May 23 18:46:19 2021: cdo -L -remap,/tmp/nctoolkitibcph...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

In [ ]:
nct.options(lazy=True)

In [ ]:
two = xr.concat(list_datasets, dim = 'time')

In [ ]:
two.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMORPH_regridded.nc')

In [ ]:
list_datasets[1]

FileNotFoundError: ignored

<xarray.Dataset>
Dimensions:        (lat: 285, lon: 245, time: 1)
Coordinates:
  * time           (time) datetime64[ns] 2016-01-02
  * lon            (lon) float32 28.5 28.55 28.6 28.65 ... 40.55 40.6 40.65 40.7
  * lat            (lat) float32 -10.7 -10.65 -10.6 -10.55 ... 3.35 3.4 3.45 3.5
Data variables:
    Precipitation  (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Sun May 23 14:51:05 2021: cdo -L -remap,/tmp/nctoolkitibcph...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

In [ ]:
two.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMORPH_regridded.nc')

In [ ]:
dataset['time']  = [pd.to_datetime('20160101')]

In [ ]:
pd.to_datetime('20160101')

Timestamp('2016-01-01 00:00:00')

In [ ]:
two.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CMORPH_regridded.nc')

In [ ]:
one = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CMORPH/daily_agg/daily_aggCMORPH_V1.0_ADJ_8km-30min_' + date_range[0] + '.nc')

In [ ]:
one.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])

In [ ]:
list_datasets = []
for date in date_range:
  dataset = nct.open


In [ ]:
! cd gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors
! ncatted -a coordinates,sst,c,c,"lon lat" Kopie_daily_aggCMORPH_V1.0_ADJ_8km-30min_20201231.nc


ncatted: ERROR file Kopie_daily_aggCMORPH_V1.0_ADJ_8km-30min_20201231.nc not found. It does not exist on the local filesystem, nor does it match remote filename patterns (e.g., http://foo or foo.bar.edu:file), nor did NCO detect a remote High Performance Storage System (HPSS) accessible via the 'hsi' command.
ncatted: HINT file-not-found errors usually arise from filename typos, incorrect paths, missing files, or capricious gods. Please verify spelling and location of requested file.


In [ ]:
!apt-get install nco

In [ ]:
!ncatted -a coordinates,sst,c,c,"lon lat" gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/Kopie_daily_aggCMORPH_V1.0_ADJ_8km-30min_20201231.nc

ncatted: ERROR File contains no variables or groups that match name sst so attribute coordinates cannot be changed


In [ ]:
one = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/CMORPH_daily/daily_aggCMORPH_V1.0_ADJ_8km-30min_20160101.nc')

In [ ]:
one.rename({'Longitude':'lon', 'Latitude':'lat', 'precip':'Precipitation'})
one.set_units({'lon':'degrees_east', 'lat':'degrees_north','Precipitation':'mm'})

In [ ]:
one.to_xarray()

<xarray.Dataset>
Dimensions:        (lat: 285, lon: 245, time: 1)
Coordinates:
  * time           (time) float64 0.0
  * lon            (lon) float32 28.5 28.55 28.6 28.65 ... 40.55 40.6 40.65 40.7
  * lat            (lat) float32 -10.7 -10.65 -10.6 -10.55 ... 3.35 3.4 3.45 3.5
Data variables:
    Precipitation  (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Sun May 23 10:46:31 2021: cdo -L -remap,/tmp/nctoolkitxzamu...
    Conventions:  CF-1.6
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...